<h1>The 8-puzzle</h1>

Before you turn in this assignment, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then run the test cells for each of the questions you have answered.  Note that a grade of 3 for the A* implementation requires all tests in the "Basic Functionality" section to be passed.  The test cells pass if they execute with no errors (i.e. all the assertions are passed).

Make sure you fill in any place that says `YOUR CODE HERE`.  Be sure to remove the `raise NotImplementedError()` statements as you implement your code - these are simply there as a reminder if you forget to add code where it's needed.

---

<h1>
Question 1    
</h1>
Define your <code>PuzzleNode</code> class below.  Ensure that you include all attributes that you need to implement an A* search.  If you wish, you can even include member functions, such as a function to generate successor states.  Alternatively, you can code up this functionality later in the <code>solvePuzzle</code> function.

In [1]:
# Import any packages you need here
# Also define any variables as needed

# YOUR CODE HERE (OPTIONAL)
import copy
#Now, define the class PuzzleNode:
class PuzzleNode:
    #Now, define the class PuzzleNode:
    
    """
    Class PuzzleNode: Provides a structure for performing A* search for the n^2-1 puzzle
    """
    def __init__(self,state,gval,fval,parent):
        
        self.state=state
        self.gval=gval
        self.fval =fval
        self.parent = parent
        self.pruned=False
        
    # Comparison function based on f cost
    def __lt__(self,other):
        return self.fval < other.fval

    '''# Convert self.state to strings
    def __str__(self):
        return str(self.state)'''
    def __str__(self):
        s =''
        for i in self.state:
            for j in i:      
                s += str(j)
                s += ' '
            s+= '\n'
        return s
    #generates child according to the number of possible moves
    #code adapted from https://github.com/murtaza98/8-puzzle-problem
    def create_child(self,move):
        i = 0
        j = 0
        found = False
        #copys the current state to create child states according to the moves available.
        next_state = copy.deepcopy(self.state)
        #self.gval=self.gval+1
        for i in range(0, len(self.state)):
            for j in range(0, len(self.state)):
                if next_state[i][j] == 0:
                    found = True
                    break
            if found:
                break
        if move == "left":
            if j == 0:
                return -1
            else:
                next_state[i][j] = next_state[i][j - 1]
                next_state[i][j - 1] = 0
                return next_state
        if move == "right":
            if j == len(self.state)-1:
                return -1
            else:
                next_state[i][j] = next_state[i][j + 1]
                next_state[i][j + 1] = 0
                return next_state
        if move == "up":
            if i == 0:
                return -1
            else:
                next_state[i][j] = next_state[i - 1][j]
                next_state[i - 1][j] = 0
                return next_state

        if move == "down":
            if i == len(self.state)-1:
                return -1
            else:
                next_state[i][j] = next_state[i + 1][j]
                next_state[i + 1][j] = 0
                return next_state
    def expand(self):
        actions=['up','down','left','right']
        child=[]
        for move in actions:
            child.append(self.create_child(move))
        #removes impossible moves with -1
        child[:] = (value for value in child if value != -1)
        return child
    #Funtion to create the goal state
    def goal_achieved(self):
        final_state=[[0 for i in range(len(self.state))] for j in range(len(self.state))]
        start_no=0
        for i in range(len(self.state)):
            for j in range(len(self.state)):
                final_state[i][j]=start_no
                start_no +=1
        return final_state
  

In [2]:
#check if expanding is happening in the right manner.
working_initial_state_15_puzzle = [[7,5,6],[2,4,3],[8,1,0]]
A=PuzzleNode(working_initial_state_15_puzzle, 0,0 ,parent = None)
A.expand()

[[[7, 5, 6], [2, 4, 0], [8, 1, 3]], [[7, 5, 6], [2, 4, 3], [8, 0, 1]]]

<h1>
Question 2    
</h1>
Define your heuristic functions using the templates below.  Ensure that you extend the <code>heuristics</code> list to include all the heuristic functions you implement.  Note that state will be given as a list of lists, so ensure your function accepts this format.  You may use packages like numpy if you wish within the functions themselves.

In [3]:
# Add any additional code here (e.g. for the memoization extension)

# YOUR CODE HERE (OPTIONAL)

# Misplaced tiles heuristic
def h1(state):
    
    """
    This function returns the number of misplaced tiles, given the state
    Input:
        -state: the puzzle state as a list of lists
    Output:
        -h: the number of misplaced tiles
    We simply calculate the goal postion for our current element and check it against the current state 
    for which if a displacement/wrong positioning exists we update our dispalcement counter by 1.
    """
    dis=0
    for i in range(len(state)):
        for j in range(len(state)):
            if state[i][j] !=0:
                goal_row,goal_col = int(state[i][j]//len(state)),state[i][j] % len(state)
                if state[i][j] != state[goal_row][goal_col]:
                    dis += 1
    return dis
    
# Manhattan distance heuristic
def h2(state):
    
    """
    This function returns the number of manhattan distance, given the board state
    Input:
        -state: the board state as a list of lists
    Output:
        -h:manhattan distance
    We simply calculate the goal postion for our current element and check it against the current state positions
    for which we calculate the manhattan displacement.
    """
    manhattan_distance=0
    for i in range(len(state)):
        for j in range(len(state)):
            if state[i][j] != 0:
                #finds the goal or correct coordinate position of the current tile.
                    x = state[i][j]%len(state)
                    y = state[i][j]//len(state)
                    #finds the absolute difference between goal and current position coordinates to give manhattan distance.
                    manhattan_distance += abs(x - j) + abs(y - i)       
    return manhattan_distance
    
# Extra heuristic for the extension.  If implemented, modify the function below
def h3(state):
    """
    This function returns a heuristic that dominates the Manhattan distance, given the board state
    Input:
        -state: the board state as a list of lists
    Output:
        -h: the Heuristic distance of the state from its solved configuration
    """
    return 0

# If you implement more than 3 heuristics, then add any extra heuristic functions onto the end of this list.
heuristics = [h1, h2, h3]

In [4]:
# Add any additional code here (e.g. for the memoization extension)

#memoized code
'''
For memoization we will use a function that will have a dictionary to store the function results for retrieval.Each 
time a new result that is not in our memoization dictionary is encountered the memoization function is called and the
new value stored.
Code adapted from https://python-course.eu/advanced-python/memoization-decorators.php
'''
def memoize(f):
    memo = {}#dictonary to store function values
    def helper(state):
        if str(state) not in memo:            
            memo[str(state)] = f(state)
        return memo[str(state)]
    return helper


# Misplaced tiles heuristic
#memoization python decorator
@memoize
def h1(state):
    
    """
    This function returns the number of misplaced tiles, given the board state
    Input:
        -state: the board state as a list of lists
    Output:
        -h: the number of misplaced tiles
    We simply calculate the goal postion for our current element and check it against the current state 
    for which if a displacement/wrong positioning exists we update our counter by 1.
    """
    dis=0
    for i in range(len(state)):
        for j in range(len(state)):
            if state[i][j] !=0:
                goal_row,goal_col = int(state[i][j]//len(state)),state[i][j] % len(state)
                if state[i][j] != state[goal_row][goal_col]:
                    dis += 1
    return dis
    
# Manhattan distance heuristic
@memoize
def h2(state):
    
    """
    This function returns the number of manhattan distance, given the board state
    Input:
        -state: the board state as a list of lists
    Output:
        -h:manhattan distance
    We simply calculate the goal postion for our current element and check it against the current state positions
    for which we calculate the manhattan displacement.
    """
    manhattan_distance=0
    for i in range(len(state)):
        for j in range(len(state)):
            if state[i][j] != 0:
                #finds the position of the current tile in coordinates
                    x = state[i][j]%len(state)
                    y = state[i][j]//len(state)
                    #finds the absolute difference between goal and current position
                    manhattan_distance += abs(x - j) + abs(y - i)       
    return manhattan_distance
    
#Extra heuristic for the extension.  If implemented, modify the function below
@memoize
def h3(state):
    """
    This function returns a heuristic that dominates the Manhattan distance, given the board state
    Input:
        -state: the board state as a list of lists
    Output:
        -h: the Heuristic distance of the state from its solved configuration
    """
    return 0

# If you implement more than 3 heuristics, then add any extra heuristic functions onto the end of this list.
heuristics_memoized = [h1, h2, h3]


heuristics_memoized[0]([[1,2,0],[3,4,5],[6,7,8]])

2

<h1>
Question 3    
</h1>
Code up your A* search using the SolvePuzzle function within the template below.  Please do not modify the function header, otherwise the automated testing will fail.  You may define other functions or import packages as needed in this cell or by adding additional cells.

In [5]:

def correct_state(state):
    """
    This function returns if the board state is of the right dimensions and has n-1 unique elements in it.
    Input:
        -state: the board state as a list of lists
    Output:
        -h: boolean value False if the board is not in the correct state and True if in correct state.
    """
    for i in range(len(state)):
        #checks if the state is in the right dimensions
        if len(state) != len(state[i]):   
            return False
    #check if our states contains all elements for n-1  
    correct_state= [i for i in range(len(state)**2)]
    for i in range(len(state)):
        for j in range(len(state[0])):
            value = state[i][j]
            if value in correct_state:
                #checks if the values in our state are all in the correct state list and have right frequency
                correct_state.remove(value)
            else:
                return False
    return True


state=[[1,2,6,3],[0,9,5,7],[4,13,10,11],[8,12,14,15]]
print(correct_state(state)) 

True


In [6]:
def solvable(state): 
    """
    Adapted from https://datawookie.dev/blog/2019/04/sliding-puzzle-solvable/
    The code works by the idea that there exists inversion for puzzles.Inversion mean that the pair of tiles are not 
    in the correct order.For example in a tile [[2,1,0],[3,4,5],[6,7,8]] there exist one inversion in the 2 and 1 
    elements.The number of inversions is then used to determine whether a puzzle is solvable or not  when we consider 
    the polarity of the inversions.For solvable tiles they even polarity and usolvale tiles have uneven polarity. 
    """
    count = 0
    for i in range(len(state)):
        for j in range(i+1, len(state)):
            if state[j] and state[i] and state[i] > state[j]:
                count += 1
    if count % 2 != 0:
        return False
    else:
        return True
    
solvable([[2,3,7],[1,8,0],[6,5,4]])

False

In [7]:
# Import any packages or define any helper functions you need here
 

# Import any packages or define any helper functions you need here

# YOUR CODE HERE (OPTIONAL)
# Main solvePuzzle function.
def solvePuzzle(state, heuristic):
    """This function should solve the n**2-1 puzzle for any n > 2 (although it may take too long for n > 4)).
    Inputs:
        -state: The initial state of the puzzle as a list of lists
        -heuristic: a handle to a heuristic function.  Will be one of those defined in Question 2.
    Outputs:
        -steps: The number of steps to optimally solve the puzzle (excluding the initial state)
        -exp: The number of nodes expanded to reach the solution
        -max_frontier: The maximum size of the frontier over the whole search
        -opt_path: The optimal path as a list of list of lists.  That is, opt_path[:,:,i] should give a list of lists
                    that represents the state of the board at the ith step of the solution.
        -err: An error code.  If state is not of the appropriate size and dimension, return -1.  For the extention task,
          if the state is not solvable, then return -2
    """
    
    max_frontier = 0 
    exp = 0     
    steps = 0
    opt_path = []
    err=0
    #conditional to check if the current state is of correct dimensions and elements and solvable
    if not correct_state(state): 
        err=-1
        return steps, exp, max_frontier, opt_path, err


    
    gval = 0 
    start = PuzzleNode(state, gval, fval = heuristic(state)+gval ,parent = None)
    #we use a priority queue to store our child/successor nodes in order of fval where we retrieve the smallest fval
    from queue import PriorityQueue
    # Dictionary with current cost to reach all visited nodes
    costs_db={}
    costs_db[str(start.state)] = start
    frontier = PriorityQueue()
    #we have inserted the first state for initialization
    frontier.put(start)
    
    
    #while we have a state explore all possible states 
    while not frontier.empty(): 
      
      #gets node with least fval and if goal we terminate
      current_node = frontier.get() 
      if current_node.state == current_node.goal_achieved(): break
      if current_node.pruned: continue
 
    
      #creates child nodes according to possible number of moves
      children = current_node.expand() 
    
      for child in children:
        #adds up gval to get to a child which is 1 for each expansion
        gval = current_node.gval + 1 
        
        if (str(child)) in costs_db:
          hval = costs_db[str(child)].fval - costs_db[str(child)].gval
          fval = hval + gval
          
          if costs_db[str(child)].gval > gval:
                # Skip if this node has been marked for pruning
                costs_db[str(child)].pruned = True
          else: 
            #skips to the next iteration of the loop to loop over all the children nodes
            continue 
        else: 
          fval = heuristic(child) + gval
        
        #create a new node to be explored and expanded and adds it to dictionary
        next_node = PuzzleNode(child,gval,fval,current_node) 
        frontier.put(next_node)  
        costs_db[str(child)] = next_node
        
        #gets the number of explored and expaned nodes and increments counter by one each time we explore another node
        exp = exp + 1

      #gets the number of explored nodes 
      max_frontier = max(max_frontier, len(frontier.queue))
    

    
 
    
    #gets our optimal list and reverses it to get the path from initial state to goal state
    opt_path = [current_node.state]
    while current_node.parent:
      steps += 1 
      opt_path.append(current_node.parent.state)
      current_node = current_node.parent
    #optimal path is reversed to display in order from start to end
    opt_path = opt_path[::-1]
    
              
    #Final value is returned. 
    return steps, exp, max_frontier, opt_path, err

solvePuzzle([[1,2,0],[3,4,5],[6,7,8]], heuristics[0])
    

(2,
 4,
 3,
 [[[1, 2, 0], [3, 4, 5], [6, 7, 8]],
  [[1, 0, 2], [3, 4, 5], [6, 7, 8]],
  [[0, 1, 2], [3, 4, 5], [6, 7, 8]]],
 0)

<h1>Extension Questions</h1>

The extensions can be implemented by modifying the code from Q2-3 above appropriately.

1. <b>Initial state solvability:</b>  Modify your SolvePuzzle function code in Q3 to return -2 if an initial state is not solvable to the goal state.
2. <b>Extra heuristic function:</b> Add another heuristic function (e.g. pattern database) that dominates the misplaced tiles and Manhattan distance heuristics to your Q2 code.
3. <b>Memoization:</b>  Modify your heuristic function definitions in Q2 by using a Python decorator to speed up heuristic function evaluation

There are test cells provided for extension questions 1 and 2.

<h1>Basic Functionality Tests</h1>
The cells below contain tests to verify that your code is working properly to be classified as basically functional.  Please note that a grade of <b>3</b> on #aicoding and #search as applicable for each test requires the test to be successfully passed.  <b>If you want to demonstrate some other aspect of your code, then feel free to add additional cells with test code and document what they do.<b>

In [8]:
## Test for state not correctly defined

incorrect_state = [[0,1,2],[2,3,4],[5,6,7]]
_,_,_,_,err = solvePuzzle(incorrect_state, lambda state: 0)
assert(err == -1)

In [9]:
## Heuristic function tests for misplaced tiles and manhattan distance

# Define the working initial states
working_initial_states_8_puzzle = ([[2,3,7],[1,8,0],[6,5,4]], [[7,0,8],[4,6,1],[5,3,2]], [[5,7,6],[2,4,3],[8,1,0]])

# Test the values returned by the heuristic functions
h_mt_vals = [7,8,7]
h_man_vals = [15,17,18]

for i in range(0,3):
    h_mt = heuristics[0](working_initial_states_8_puzzle[i])
    h_man = heuristics[1](working_initial_states_8_puzzle[i])
    assert(h_mt == h_mt_vals[i])
    assert(h_man == h_man_vals[i])


In [10]:
## A* Tests for 3 x 3 boards
## This test runs A* with both heuristics and ensures that the same optimal number of steps are found
## with each heuristic.

# Optimal path to the solution for the first 3 x 3 state
opt_path_soln = [[[2, 3, 7], [1, 8, 0], [6, 5, 4]], [[2, 3, 7], [1, 8, 4], [6, 5, 0]], 
                 [[2, 3, 7], [1, 8, 4], [6, 0, 5]], [[2, 3, 7], [1, 0, 4], [6, 8, 5]], 
                 [[2, 0, 7], [1, 3, 4], [6, 8, 5]], [[0, 2, 7], [1, 3, 4], [6, 8, 5]], 
                 [[1, 2, 7], [0, 3, 4], [6, 8, 5]], [[1, 2, 7], [3, 0, 4], [6, 8, 5]], 
                 [[1, 2, 7], [3, 4, 0], [6, 8, 5]], [[1, 2, 0], [3, 4, 7], [6, 8, 5]], 
                 [[1, 0, 2], [3, 4, 7], [6, 8, 5]], [[1, 4, 2], [3, 0, 7], [6, 8, 5]], 
                 [[1, 4, 2], [3, 7, 0], [6, 8, 5]], [[1, 4, 2], [3, 7, 5], [6, 8, 0]], 
                 [[1, 4, 2], [3, 7, 5], [6, 0, 8]], [[1, 4, 2], [3, 0, 5], [6, 7, 8]], 
                 [[1, 0, 2], [3, 4, 5], [6, 7, 8]], [[0, 1, 2], [3, 4, 5], [6, 7, 8]]]

astar_steps = [17, 25, 28]
for i in range(0,3):
    steps_mt, expansions_mt, _, opt_path_mt, _ = solvePuzzle(working_initial_states_8_puzzle[i], heuristics[0])
    steps_man, expansions_man, _, opt_path_man, _ = solvePuzzle(working_initial_states_8_puzzle[i], heuristics[1])
    # Test whether the number of optimal steps is correct and the same
    assert(steps_mt == steps_man == astar_steps[i])
    # Test whether or not the manhattan distance dominates the misplaced tiles heuristic in every case
    assert(expansions_man < expansions_mt)
    # For the first state, test that the optimal path is the same
    if i == 0:
        assert(opt_path_mt == opt_path_soln)


In [11]:
## A* Test for 4 x 4 board
## This test runs A* with both heuristics and ensures that the same optimal number of steps are found
## with each heuristic.

working_initial_state_15_puzzle = [[1,2,6,3],[0,9,5,7],[4,13,10,11],[8,12,14,15]]
steps_mt, expansions_mt, _, _, _ = solvePuzzle(working_initial_state_15_puzzle, heuristics[0])
steps_man, expansions_man, _, _, _ = solvePuzzle(working_initial_state_15_puzzle, heuristics[1])
# Test whether the number of optimal steps is correct and the same
assert(steps_mt == steps_man == 9)
print(steps_man)
# Test whether or not the manhattan distance dominates the misplaced tiles heuristic in every case
assert(expansions_mt >= expansions_man)

9
